# ARD-LoRA Training with Uncertainty Evaluation on Google Colab

This notebook trains ARD-LoRA (Automatic Relevance Determination LoRA) models with comprehensive uncertainty evaluation using:

🎯 **Key Features:**
- **Bayesian-PEFT Dataset Integration**: Compatible with https://github.com/Wang-ML-Lab/bayesian-peft
- **ARD-LoRA for LLaMA2-7B**: Injects ProbLoRA into q/k/v/o projections
- **Uncertainty Evaluation**: ACC, ECE, NLL metrics after each epoch
- **Google Drive Caching**: Persistent dataset storage across sessions
- **Complete Callback System**: Prior estimation, latent plotting, evaluation

📊 **Training Pipeline:**
1. Mount Google Drive for persistent caching
2. Install dependencies and setup environment
3. Load LLaMA2-7B with ARD-LoRA injection
4. Load Bayesian-PEFT datasets with caching
5. Train with uncertainty evaluation after each epoch
6. Save results and model checkpoints

⚡ **Expected Runtime**: ~2-3 hours for 3 epochs with T4 GPU

## Step 1: Mount Google Drive & Setup Environment

In [ ]:
# Mount Google Drive for persistent storage
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Verify mount
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Google Drive mounted successfully!")
    print(f"📁 Available space: {os.statvfs('/content/drive/MyDrive').f_bavail * os.statvfs('/content/drive/MyDrive').f_frsize // (1024**3)} GB")
else:
    print("❌ Google Drive mount failed!")

# Create ARD-LoRA directories
ard_lora_dir = "/content/drive/MyDrive/ARD_LoRA_Training"
cache_dir = f"{ard_lora_dir}/DataCache"
results_dir = f"{ard_lora_dir}/Results"

os.makedirs(ard_lora_dir, exist_ok=True)
os.makedirs(cache_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

print(f"📂 ARD-LoRA directory: {ard_lora_dir}")
print(f"💾 Cache directory: {cache_dir}")
print(f"📊 Results directory: {results_dir}")

## Step 2: Install Dependencies

In [ ]:
# Install required packages not available in default Colab
!pip install -q transformers>=4.21.0
!pip install -q torch>=2.0.0
!pip install -q datasets>=2.5.0
!pip install -q accelerate>=0.20.0
!pip install -q peft>=0.4.0
!pip install -q bitsandbytes>=0.41.0
!pip install -q tensorboard
!pip install -q scikit-learn
!pip install -q matplotlib seaborn
!pip install -q tqdm

print("✅ All dependencies installed successfully!")

## Step 3: Clone ARD-LoRA Repository

In [ ]:
import os
import sys

# Clone the ARD-LoRA repository
repo_dir = "/content/ARD-LoRA-Data-CLM"
if not os.path.exists(repo_dir):
    !git clone https://github.com/Surojit-Utah/ARD-LoRA-Data.git /content/ARD-LoRA-Data-temp
    # Copy the CLM-specific implementation
    !cp -r /content/ARD-LoRA-Data-temp/ARD-LoRA-Data-CLM /content/
    !rm -rf /content/ARD-LoRA-Data-temp
    print("✅ Repository cloned successfully!")
else:
    print("✅ Repository already exists!")

# Add to Python path
if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

# Change working directory
os.chdir(repo_dir)
print(f"📂 Working directory: {os.getcwd()}")

# Verify key files exist
key_files = [
    "run_training_cached.py",
    "model/model_llama.py", 
    "trainer/trainer_clm.py",
    "dataloader/bayesian_peft_cached.py",
    "evaluate/uncertainty_metrics.py"
]

for file in key_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - MISSING!")

## Step 4: Configure Training Parameters

In [ ]:
# Configure training parameters for Colab
import yaml
import json

# Colab-optimized configuration
colab_config = {
    "defaults": {
        "runId": 1,
        # Cache configuration for Google Drive
        "cache_root": cache_dir,
        "google_drive_cache": cache_dir,
        "use_google_drive": True,
        
        # Model configuration optimized for Colab T4
        "learning_rate": 2e-5,
        "lr_scheduler_type": "linear",
        "warmup_ratio": 0.03,
        "weight_decay": 0.0,
        "train_epochs": 3,
        "rank": 32,  # Reduced for Colab memory
        "max_len": 1024,  # Reduced for Colab memory
        "batch_size": 2,  # Small batch size for T4
        "gradient_accumulation_steps": 16,  # Compensate for small batch
        "fp16": True,
        "load_in_4bit": True,  # Essential for Colab memory
        "prior_var": 1.0,
        "kl_loss_beta": 0.01,
        
        # ARD and uncertainty configuration
        "ard_prior_samples": 500,  # Reduced for speed
        "ard_prior_ratio": 0.5,
        "uncertainty_eval_samples": 500,  # Reduced for speed
        "uncertainty_n_bins": 15,
        
        # Callback configuration
        "enable_callbacks": True,
        "enable_plotting": True,
        "enable_resampling": False,  # Disabled for simplicity
        "plot_start_epoch": 2,
        "plot_interval": 1,
        
        # Logging
        "report_to": ["tensorboard"],
        "logging_steps": 10,
        "eval_steps": None,
        "save_steps": None,
        "evaluation_strategy": "epoch",
        "save_strategy": "epoch"
    },
    
    "models": {
        "LLaMA2-7B": {
            "model_name_or_path": "meta-llama/Llama-2-7b-hf",
            "tokenizer_name": "meta-llama/Llama-2-7b-hf",
            "train_type": "causal_lm",
            "load_in_4bit": True,
            "defaults": {
                "rank": 32,
                "max_len": 1024,
                "batch_size": 2,
                "kl_loss_beta": 0.005
            }
        }
    },
    
    "datasets": {
        "BayesianPEFT": {
            "dataset_type": "bayesian_peft",
            "dataset_name": "alpaca",  # Start with Alpaca
            "dataset_class": "S2SDataset",
            "repo_url": "https://github.com/Wang-ML-Lab/bayesian-peft",
            "cache_root": cache_dir,
            "max_len": 1024,
            "num_labels": 0
        }
    }
}

# Save configuration
os.makedirs("config", exist_ok=True)
with open("config/colab_config.yaml", "w") as f:
    yaml.dump(colab_config, f, default_flow_style=False, indent=2)

print("✅ Colab configuration saved!")
print(f"📊 Training Configuration:")
print(f"   - Model: LLaMA2-7B (4-bit quantized)")
print(f"   - Dataset: Alpaca (Bayesian-PEFT compatible)")
print(f"   - Epochs: {colab_config['defaults']['train_epochs']}")
print(f"   - Rank: {colab_config['defaults']['rank']}")
print(f"   - Batch Size: {colab_config['defaults']['batch_size']}")
print(f"   - Max Length: {colab_config['defaults']['max_len']}")
print(f"   - Cache: {cache_dir}")

## Step 5: Update Configuration Loading

In [ ]:
# Override the config loading to use our Colab configuration
import yaml

# Load our Colab configuration
with open("config/colab_config.yaml", "r") as f:
    COLAB_CONFIG = yaml.safe_load(f)

# Override the CONFIG in config/__init__.py
import config
config.CONFIG = COLAB_CONFIG

print("✅ Configuration updated for Colab!")
print(f"🔧 Using custom Colab configuration with Google Drive caching")

## Step 6: Import ARD-LoRA Components

In [ ]:
# Import all ARD-LoRA components
import torch
import numpy as np
from pathlib import Path
import gc
import warnings
warnings.filterwarnings('ignore')

# Import ARD-LoRA components
from model.model_llama import inject_problora_llama, ProbLoRALayer
from trainer.trainer_clm import ARDCLMTrainer, build_clm_trainer, create_ard_callbacks
from dataloader.bayesian_peft_cached import load_bayesian_peft_with_caching
from evaluate.uncertainty_metrics import UncertaintyEvaluator
from utils.io import get_output_dirs, free_memory

# Import transformers
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig,
    TrainingArguments
)

print("✅ All ARD-LoRA components imported successfully!")

# Check GPU availability
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"🚀 GPU: {gpu_name} ({gpu_memory:.1f} GB)")
else:
    print("⚠️ No GPU available - training will be very slow!")

## Step 7: Load LLaMA2-7B Model with ARD-LoRA Injection

In [ ]:
# Load LLaMA2-7B with 4-bit quantization and inject ARD-LoRA
def load_ard_lora_model():
    print("🔄 Loading LLaMA2-7B model...")
    
    # 4-bit quantization for Colab memory efficiency
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    
    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-2-7b-hf",
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    print("✅ Model loaded successfully!")
    
    # Inject ARD-LoRA into all attention projections
    print("🔄 Injecting ARD-LoRA layers...")
    model = inject_problora_llama(
        model,
        rank=32,           # Colab-optimized rank
        scaling=1.0,
        prior_var=1.0,
        num_tokens=1024,   # Colab-optimized sequence length
        ard_prior_samples=500
    )
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"✅ ARD-LoRA injection completed!")
    print(f"📊 Model Statistics:")
    print(f"   - Total parameters: {total_params:,}")
    print(f"   - Trainable parameters: {trainable_params:,}")
    print(f"   - Trainable percentage: {100 * trainable_params / total_params:.2f}%")
    
    # Count ARD-LoRA layers
    ard_layers = 0
    for name, module in model.named_modules():
        if isinstance(module, ProbLoRALayer):
            ard_layers += 1
    print(f"   - ARD-LoRA layers: {ard_layers}")
    
    return model, tokenizer

# Load the model
model, tokenizer = load_ard_lora_model()

# Clear memory
gc.collect()
torch.cuda.empty_cache()
print("🧹 Memory cleared")

## Step 8: Load Bayesian-PEFT Dataset with Caching

In [ ]:
# Load Bayesian-PEFT compatible dataset with Google Drive caching
def load_dataset_with_caching():
    print("🔄 Loading Bayesian-PEFT dataset with caching...")
    
    # Configuration for dataset loading
    dataset_config = {
        "dataset_name": "alpaca",
        "max_len": 1024,
        "cache_root": cache_dir,
        "batch_size": 2,
        "num_labels": 0
    }
    
    try:
        # Load with Bayesian-PEFT compatibility
        train_ds, val_ds, tokenizer_updated = load_bayesian_peft_with_caching(
            dataset_name="alpaca",
            tokenizer_name="meta-llama/Llama-2-7b-hf",
            config=dataset_config,
            cache_root=cache_dir
        )
        
        print("✅ Dataset loaded successfully!")
        print(f"📊 Dataset Statistics:")
        print(f"   - Training samples: {len(train_ds) if train_ds else 0}")
        print(f"   - Validation samples: {len(val_ds) if val_ds else 0}")
        print(f"   - Cache location: {cache_dir}")
        
        # Check cache size
        cache_size = sum(f.stat().st_size for f in Path(cache_dir).glob('**/*') if f.is_file()) / (1024**2)
        print(f"   - Cache size: {cache_size:.1f} MB")
        
        return train_ds, val_ds
        
    except Exception as e:
        print(f"❌ Dataset loading failed: {e}")
        print("🔄 This is expected on first run - datasets will be downloaded and cached")
        raise

# Load datasets
try:
    train_dataset, val_dataset = load_dataset_with_caching()
except Exception as e:
    print(f"⚠️ Dataset loading issue: {e}")
    print("Please check the error and re-run if needed")

## Step 9: Create ARD-LoRA Trainer with Callbacks

In [ ]:
# Create enhanced ARD-LoRA trainer with uncertainty evaluation
def create_ard_trainer(model, tokenizer, train_ds, val_ds):
    print("🔄 Creating ARD-LoRA trainer...")
    
    # Setup output directories
    output_dir = f"{results_dir}/ARD_LoRA_LLaMA2-7B_Alpaca_Colab"
    os.makedirs(output_dir, exist_ok=True)
    
    # Training configuration optimized for Colab
    training_config = {
        "train_epochs": 3,
        "batch_size": 2,
        "gradient_accumulation_steps": 16,
        "learning_rate": 2e-5,
        "weight_decay": 0.0,
        "lr_scheduler_type": "linear",
        "warmup_ratio": 0.03,
        "fp16": True,
        "beta": 0.01,  # KL regularization strength
        
        # ARD configuration
        "ard_prior_ratio": 0.5,
        "ard_prior_samples": 500,
        "uncertainty_eval_samples": 500,
        "uncertainty_n_bins": 15,
        
        # Callback configuration
        "enable_callbacks": True,
        "enable_plotting": True,
        "enable_resampling": False,
        "plot_start_epoch": 2,
        "plot_interval": 1,
        
        # Logging
        "report_to": ["tensorboard"],
        "logging_steps": 5,
        "evaluation_strategy": "epoch",
        "save_strategy": "epoch"
    }
    
    # Create trainer with all components
    trainer = build_clm_trainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        cfg=training_config,
        output_dir=output_dir,
        ard_prior_ratio=training_config["ard_prior_ratio"],
        enable_callbacks=training_config["enable_callbacks"]
    )
    
    print("✅ ARD-LoRA trainer created successfully!")
    print(f"📊 Training Configuration:")
    print(f"   - Epochs: {training_config['train_epochs']}")
    print(f"   - Batch size: {training_config['batch_size']}")
    print(f"   - Gradient accumulation: {training_config['gradient_accumulation_steps']}")
    print(f"   - Learning rate: {training_config['learning_rate']}")
    print(f"   - KL beta: {training_config['beta']}")
    print(f"   - Output directory: {output_dir}")
    
    # Check callback registration
    if hasattr(trainer, 'callback_handler') and trainer.callback_handler.callbacks:
        callback_names = [type(cb).__name__ for cb in trainer.callback_handler.callbacks]
        print(f"   - Callbacks: {', '.join(callback_names)}")
    
    return trainer, output_dir

# Create trainer
if 'train_dataset' in locals() and 'val_dataset' in locals():
    trainer, output_directory = create_ard_trainer(model, tokenizer, train_dataset, val_dataset)
    print("🎯 Ready for training!")
else:
    print("⚠️ Datasets not loaded - please run Step 8 first")

## Step 10: Start ARD-LoRA Training with Uncertainty Evaluation

In [ ]:
# Start comprehensive ARD-LoRA training
def start_training():
    print("🚀 Starting ARD-LoRA training with uncertainty evaluation...")
    print("=" * 80)
    
    try:
        # Initial evaluation
        if val_dataset:
            print("🔄 Running initial evaluation...")
            initial_metrics = trainer.evaluate()
            print(f"📊 Initial eval loss: {initial_metrics.get('eval_loss', 'N/A'):.4f}")
        
        # Start training with automatic callbacks
        print("\n🏋️ Training started...")
        print("Expected behavior each epoch:")
        print("  1. PriorEstimationCallback: Estimate ARD priors")
        print("  2. Training steps with KL regularization")
        print("  3. UncertaintyEvaluationCallback: Compute ACC, ECE, NLL")
        print("  4. LatentPlotCallback: Generate plots (epochs 2+)")
        print("  5. Model checkpoint saving")
        print("\n" + "=" * 80)
        
        # Train the model
        training_results = trainer.train()
        
        print("\n" + "=" * 80)
        print("🎉 Training completed successfully!")
        print(f"📊 Final training loss: {training_results.training_loss:.4f}")
        
        # Display uncertainty evolution
        if hasattr(trainer, 'uncertainty_results') and trainer.uncertainty_results:
            print(f"\n📈 Uncertainty Evolution Across Epochs:")
            print(f"{'Epoch':<8} {'Accuracy':<10} {'ECE':<10} {'NLL':<10}")
            print("-" * 40)
            for result in trainer.uncertainty_results:
                epoch = result.get('epoch', '?')
                acc = result.get('accuracy', 0)
                ece = result.get('ece', 0)
                nll = result.get('nll', 0)
                print(f"{epoch:<8} {acc:<10.4f} {ece:<10.4f} {nll:<10.4f}")
        
        # Save final model
        print(f"\n💾 Saving final model...")
        trainer.save_model()
        
        # Save uncertainty results
        if hasattr(trainer, 'uncertainty_results'):
            results_file = f"{output_directory}/uncertainty_evolution.json"
            import json
            with open(results_file, 'w') as f:
                json.dump(trainer.uncertainty_results, f, indent=2)
            print(f"📊 Uncertainty results saved to: {results_file}")
        
        print(f"\n✅ All results saved to: {output_directory}")
        print(f"📁 Available in Google Drive: {output_directory.replace('/content/drive/MyDrive/', '')}")
        
        return training_results
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    finally:
        # Always clean up memory
        gc.collect()
        torch.cuda.empty_cache()
        print("🧹 Memory cleaned up")

# Start training
if 'trainer' in locals():
    training_results = start_training()
else:
    print("⚠️ Trainer not created - please run previous steps first")

## Step 11: Analyze Training Results

In [ ]:
# Analyze and visualize training results
import matplotlib.pyplot as plt
import json
import pandas as pd

def analyze_results():
    print("📊 Analyzing ARD-LoRA training results...")
    
    # Load uncertainty results
    results_file = f"{output_directory}/uncertainty_evolution.json"
    if os.path.exists(results_file):
        with open(results_file, 'r') as f:
            uncertainty_results = json.load(f)
        
        # Convert to DataFrame for analysis
        df = pd.DataFrame(uncertainty_results)
        
        # Create visualizations
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('ARD-LoRA Training Results: Uncertainty Evolution', fontsize=16)
        
        # Plot 1: Accuracy evolution
        axes[0, 0].plot(df['epoch'], df['accuracy'], 'b-o', linewidth=2, markersize=8)
        axes[0, 0].set_title('Accuracy (ACC) Evolution', fontsize=14)
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Accuracy')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].set_ylim(0, 1)
        
        # Plot 2: ECE evolution
        axes[0, 1].plot(df['epoch'], df['ece'], 'r-s', linewidth=2, markersize=8)
        axes[0, 1].set_title('Expected Calibration Error (ECE)', fontsize=14)
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('ECE')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Plot 3: NLL evolution
        axes[1, 0].plot(df['epoch'], df['nll'], 'g-^', linewidth=2, markersize=8)
        axes[1, 0].set_title('Negative Log-Likelihood (NLL)', fontsize=14)
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('NLL')
        axes[1, 0].grid(True, alpha=0.3)
        
        # Plot 4: Combined normalized metrics
        # Normalize metrics for comparison
        acc_norm = df['accuracy'] / df['accuracy'].max()
        ece_norm = 1 - (df['ece'] / df['ece'].max())  # Invert ECE (lower is better)
        nll_norm = 1 - ((df['nll'] - df['nll'].min()) / (df['nll'].max() - df['nll'].min()))  # Invert NLL
        
        axes[1, 1].plot(df['epoch'], acc_norm, 'b-o', label='Accuracy (norm)', linewidth=2)
        axes[1, 1].plot(df['epoch'], ece_norm, 'r-s', label='ECE (inv norm)', linewidth=2)
        axes[1, 1].plot(df['epoch'], nll_norm, 'g-^', label='NLL (inv norm)', linewidth=2)
        axes[1, 1].set_title('Normalized Metrics Comparison', fontsize=14)
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Normalized Score')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].set_ylim(0, 1.1)
        
        plt.tight_layout()
        
        # Save plot
        plot_path = f"{output_directory}/uncertainty_evolution.png"
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"📈 Visualization saved to: {plot_path}")
        
        # Print summary statistics
        print("\n📊 Training Summary:")
        print(f"   Initial → Final Accuracy: {df['accuracy'].iloc[0]:.4f} → {df['accuracy'].iloc[-1]:.4f}")
        print(f"   Initial → Final ECE: {df['ece'].iloc[0]:.4f} → {df['ece'].iloc[-1]:.4f}")
        print(f"   Initial → Final NLL: {df['nll'].iloc[0]:.4f} → {df['nll'].iloc[-1]:.4f}")
        
        improvement_acc = df['accuracy'].iloc[-1] - df['accuracy'].iloc[0]
        improvement_ece = df['ece'].iloc[0] - df['ece'].iloc[-1]  # Lower is better
        improvement_nll = df['nll'].iloc[0] - df['nll'].iloc[-1]  # Lower is better
        
        print(f"\n📈 Improvements:")
        print(f"   Accuracy: {improvement_acc:+.4f} ({'✅ Improved' if improvement_acc > 0 else '❌ Degraded'})")
        print(f"   ECE: {improvement_ece:+.4f} ({'✅ Improved' if improvement_ece > 0 else '❌ Degraded'})")
        print(f"   NLL: {improvement_nll:+.4f} ({'✅ Improved' if improvement_nll > 0 else '❌ Degraded'})")
        
    else:
        print(f"⚠️ Results file not found: {results_file}")
    
    # Check for other artifacts
    print(f"\n📁 Training Artifacts in {output_directory}:")
    if os.path.exists(output_directory):
        for item in sorted(os.listdir(output_directory)):
            item_path = os.path.join(output_directory, item)
            if os.path.isfile(item_path):
                size_mb = os.path.getsize(item_path) / (1024**2)
                print(f"   📄 {item} ({size_mb:.1f} MB)")
            else:
                print(f"   📁 {item}/")
    
    print(f"\n✅ Analysis complete! All files available in Google Drive.")

# Run analysis
if 'output_directory' in locals():
    analyze_results()
else:
    print("⚠️ No training results to analyze - please complete training first")

## Step 12: Download Results (Optional)

In [ ]:
# Download key results for local analysis
from google.colab import files
import zipfile

def download_results():
    print("📦 Preparing results for download...")
    
    if 'output_directory' not in locals():
        print("⚠️ No training results available")
        return
    
    # Create a zip file with key results
    zip_path = "/content/ard_lora_results.zip"
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add key files
        result_files = [
            "uncertainty_evolution.json",
            "uncertainty_evolution.png", 
            "training_args.bin",
            "trainer_state.json"
        ]
        
        for filename in result_files:
            file_path = f"{output_directory}/{filename}"
            if os.path.exists(file_path):
                zipf.write(file_path, filename)
                print(f"   ✅ Added {filename}")
            else:
                print(f"   ⚠️ {filename} not found")
        
        # Add model config if available
        config_path = f"{output_directory}/config.json"
        if os.path.exists(config_path):
            zipf.write(config_path, "config.json")
            print(f"   ✅ Added config.json")
    
    print(f"📦 Results packaged in: {zip_path}")
    
    # Download the zip file
    files.download(zip_path)
    print("⬇️ Download started!")

# Uncomment to download results
# download_results()

## 🎉 Training Complete!

**Congratulations!** You have successfully trained an ARD-LoRA model with comprehensive uncertainty evaluation.

### **What Was Accomplished:**

✅ **ARD-LoRA Integration**: ProbLoRA layers injected into all LLaMA2-7B attention projections (q/k/v/o)

✅ **Bayesian-PEFT Compatibility**: Dataset loading consistent with https://github.com/Wang-ML-Lab/bayesian-peft

✅ **Uncertainty Evaluation**: ACC, ECE, and NLL computed after each epoch

✅ **ARD Prior Estimation**: Automatic relevance determination for layer importance

✅ **Google Drive Persistence**: All data and results cached for future use

✅ **Complete Callback System**: Prior estimation, latent plotting, and evaluation callbacks

### **Key Results:**
- **Model Checkpoints**: Saved after each epoch in Google Drive
- **Uncertainty Evolution**: JSON file tracking ACC/ECE/NLL across epochs  
- **Visualizations**: Plots showing uncertainty metric evolution
- **ARD Analysis**: Layer-wise relevance determination results
- **Tensorboard Logs**: Available for detailed training monitoring

### **Next Steps:**
1. **Analyze Results**: Review uncertainty evolution and model calibration
2. **Experiment with Hyperparameters**: Adjust KL beta, rank, or dataset
3. **Extended Training**: Increase epochs for further improvement
4. **Model Deployment**: Use trained model for downstream tasks

All results are automatically saved to your Google Drive and will persist across Colab sessions! 🚀